In [0]:
!pip install torch torchtext torchvision

In [0]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torchtext.vocab import GloVe
import torch.nn.functional as f

In [0]:
class BaselineQACNN(nn.Module):
  
  def __init__(self,emb_dim,dict_size,hidden_dim,ctx_window):
    super(BaselineQACNN,self).__init__()
    self.emb = nn.Embedding(dict_size,emb_dim) # with learning, try the GloVe without learning
    self.convolution_q = nn.Conv2d(emb_dim,hidden_dim,kernel_size=(emb_dim,ctx_window)) # CARE: padding??
    self.convolution_a = nn.Conv2d(emb_dim,hidden_dim,kernel_size=(emb_dim,ctx_window))
    self.h_pool = lambda t : self.horizontal_pooling(t)
    
  def forward(self,q,a):
    q_embed = self.emb(q)
    a_embed = self.emb(a)
    q_enc = F.relu(self.convolution_q(q_embed))
    a_enc = F.relu(self.convolution_a(a_embed))
    r_q = self.h_pool(q_enc)
    r_a = self.h_pool(a_enc)
    return f.cosine_similarity(r_q,r_a)
  
  # given a matrix, does a maxpool operation on the rows
  # t.view(t.size(0),-1) = flatten into a 1D tensor each outcome for each input sample
  # passing from (n,a,b) to (n,a) dimensions
  def horizontal_pooling(t):
    return f.max_pool1d(t,t.size(2)).view(t.size(0),-1)

class BaselineQAbiLSTM(nn.Module):
  
  def __init__(self,emb_dim,dict_size,single_hidden_dim):
    super(BaselineQAbiLSTM,self).__init__()
    self.emb = nn.Embedding(dict_size,emb_dim) # with learning, try the GloVe without learning
    self.bilstm_q = nn.LSTM(emb_dim,single_hidden_dim,bidirectional=True)
    self.bilstm_a = nn.LSTM(emb_dim,single_hidden_dim,bidirectional=True)
    self.h_pool = lambda t : self.horizontal_pooling(t)
    
  def forward(self,q,a):
    q_embed = self.emb(q)
    a_embed = self.emb(a)
    q_enc = F.relu(self.bilstm_q(q_embed))
    a_enc = F.relu(self.bilstm_a(a_embed))
    r_q = self.h_pool(q_enc)
    r_a = self.h_pool(a_enc)
    return f.cosine_similarity(r_q,r_a)
    
    
  # given a matrix, does a maxpool operation on the rows
  # t.view(t.size(0),-1) = flatten into a 1D tensor each outcome for each input sample
  # passing from (n,a,b) to (n,a) dimensions
  def horizontal_pooling(t):
    return f.max_pool1d(t,t.size(2)).view(t.size(0),-1)

In [0]:
model = BaselineQACNN(10,400,3,5)



---

*Draft snippets*

---

In [0]:
import numpy as np

#a = np.array([[[1,2,3],[4,5,6],[7,8,9]],[[10,11,12],[13,14,15],[16,17,18]]])
#a = np.array([[[[1],[2],[3]],[[4],[5],[6]],[[7],[8],[9]]],[[[10],[11],[12]],[[13],[14],[15]],[[16],[17],[18]]]])
a = np.array([[[[1,2,3],[4,5,6],[7,8,9]]],[[[10,11,12],[13,14,15],[16,17,18]]]])
t = torch.from_numpy(a).double()
#m = nn.MaxPool2d((1,3))
#po = f.max_pool1d(t,t.size(2))
#torch.cat(tuple(po),dim=1)
#po.view(t.size(0),-1)
c = nn.Conv2d(1,1,2).double()
c(t)


In [0]:
from torchvision import datasets,transforms
data = datasets.MNIST('.',train=True,download=True,transform= transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))]))

In [0]:
torch.device("cuda:0" if torch.cuda.is_available() else "cpu")